# Presentation Skeleton

* Goal was to see if crime rates differed between gentrified and non-gentrified neighborhoods over time, to see both if changes in crime correlate with gentrification, later we might use the analysis to compare with another city. 

* City of Chicago Data called by API (see below), looks at three years [2002, 2010, 2018], and makes a call for up to 400,000 reported cries per year. Because the API throttles API calls that are greater than 400,000 records long, the code below is broken up such that the operations for each year take place one at a time. More elegant code could abbreviate the process, but slowing down the processing speed made the API call more stable and consistent. 

* To connect crime data, with fbi code books, gentrification indeces and demographic information, the majority of the code is a series of pivot-like data manipulations, resulting in tables that provide summary counts by crime category [violent, nonviolent, white collar] and in neighborhoods that were coded [gentrified, not gentrified]. From an intial 1.2m reported crimes, our subest is closer to 300,000, including 30 of 70 community areas.

* Next we merge with demographic data to adjust count of reported crimes by population to generate a crime rate. The crime rate per neighborhood type, crime type, and year were then plotted in a series of box plots. 

#### Example Socrata Module for City of Chicago Data. https://dev.socrata.com/foundry/data.cityofchicago.org/6zsd-86xi
* I don't use the sodapy package, the code is more efficient, but I'm trying to demonstrate that I'm learning the DS_Bootcamp code

In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import warnings;
warnings.filterwarnings('ignore');

# import pandas as pd
from sodapy import Socrata

MyAppToken = 'PoLZLhceSUgObx1Disd6WIQS1'

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 MyAppToken,
                 username="stephen.monteiro@northwestern.edu",
                 password="FackeTestPw0rd")

In [2]:
#Dependencies
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pprint as pp
import os
import csv
year = [2002, 2010, 2018]


In [3]:
# fbi_codex Needed to index Crime Reports into summary categories
fbi_codex = pd.read_csv('crime_resources/fbi_codex.csv')
fbi_index = fbi_codex[['fbi_Crime_type','fbi_code_key','ds_fbi_group']]
fbi_index = fbi_index.dropna()

# gent_codex Needed to index Crime Reports into summary categories
gent_codex = pd.read_csv('../../Resources/UIC_gentrification_index_neighborhood_list_final.csv')
gent_concat = gent_codex["Type"].str.split(" - ", n = 1, expand = True) 
gent_codex["Classification"] = gent_concat[1]
gent_codex["Class_Code"] = gent_concat[0]
gent_fin = gent_codex.rename(columns={"No.": "community_area"})

# cca_codex will allow us to convert raw crime into a crime rate for 2010 and an approzimation for 2002
cca_codex = pd.read_csv('../../Resources/ReferenceCCA20102014.csv')

# cca_codex2018 will allow us to convert raw crime into a crime rate for 2018

cca_codex2018 = pd.read_csv('../../Resources/DemoData2018.csv')

# display all imported csvs so we can remember what we're working with later
display(fbi_index.head())
display(gent_fin.head())
display(cca_codex.head())
display(cca_codex2018.head())

,fbi_Crime_type,fbi_code_key,ds_fbi_group
0,Aggravated Assault,04A,Violent
1,Aggravated Battery,04B,Violent
2,Arson,9,Nonviolent
3,Burglary,5,Nonviolent
4,Criminal Sexual Abuse,17,Nonviolent


,community_area,Neighborhood,1970,1980,1990,2000,2010,1970-2010,Change,Type,Classification,Class_Code
0,77,Edgewater,7,9,9,7,9,2,No Change,1 - Upper Class,Upper Class,1
1,9,Edison Park,10,9,11,11,11,1,No Change,1 - Upper Class,Upper Class,1
2,12,Forest Glen,11,11,11,11,9,-2,No Change,1 - Upper Class,Upper Class,1
3,4,Lincoln Square,9,7,5,11,11,2,No Change,1 - Upper Class,Upper Class,1
4,13,North Park,9,11,11,11,11,2,No Change,1 - Upper Class,Upper Class,1


,GEOG,2000_POP,2010_POP,POP_CHANGE,TOT_POP,UND19,A20_34,A35_49,A50_64,A65_79,...,IND,INDperc,TRANS,TRANSperc,AG,Agperc,OPEN,OPENperc,VACANT,VACperc
0,Albany Park,57655,51542,-6113,52930,14351,14417,11908,8016,3062,...,19.929938,1.621420,414.736812,33.741335,0.0,0.0,25.293181,2.057752,6.672887,0.542880
1,Archer Heights,12644,13393,749,13971,4319,3277,2876,2058,1020,...,344.677590,26.848038,558.599207,43.511076,0.0,0.0,12.295439,0.957731,32.517216,2.532870
2,Armour Square,12032,13391,1359,13907,3037,3206,2434,2651,1639,...,18.046053,2.831079,343.648391,53.911829,0.0,0.0,15.019249,2.356231,14.366379,2.253809
3,Ashburn,39584,41081,1497,42809,12553,8473,8808,8724,3201,...,104.574805,3.362800,1064.249375,34.222945,0.0,0.0,259.867667,8.356535,83.385680,2.681424
4,Auburn Gresham,55928,48743,-7185,47611,13305,9014,8262,9515,5541,...,57.729492,2.393450,834.435659,34.595488,0.0,0.0,59.011603,2.446606,119.183176,4.941304


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,<<<<<<< HEAD:Resources/DemoData2018.csv
GEOG,2000_POP,2010_POP,TOT_POP,UND19,A20_34,A35_49,A50_64,A65_74,A75_84,OV85,MED_AGE,WHITE,HISP,BLACK,ASIAN,OTHER,POP_HH,POP_16OV,IN_LBFRC,EMP,UNEMP,NOT_IN_LBFRC,WORK_AT_HOME,TOT_COMM,DROVE_AL,CARPOOL,TRANSIT,WALK_BIKE,COMM_OTHER,AGG_TT,NO_VEH,ONE_VEH,TWO_VEH,THREEOM_VEH,POP_25OV,LT_HS,HS,SOME_COLL,ASSOC,BACH,GRAD_PROF,INC_LT_25K,INC_25_50K,INC_50_75K,INC_75_100K,INC_100_150K,INC_GT_150,MEDINC,TOT_HH,OWN_OCC_HU,RENT_OCC_HU,VAC_HU,HU_TOT,HU_SNG_DET,HU_SNG_ATT,HU_2UN,HU_3_4UN,HU_GT_5UN,MED_ROOMS,HA_AFT2000,HA_70_00,HA_40_70,HA_BEF1940,MED_HA,BR_0_1,BR_2,BR_3,BR_4,BR_5,TOT_EMP_RES,RES_NAICS1_TYPE,RES_NAICS2_TYPE,RES_NAICS3_TYPE,RES_NAICS4_TYPE,RES_NAICS5_TYPE,RES_NAICS1_COUNT,RES_NAICS2_COUNT,RES_NAICS3_COUNT,RES_NAICS4_COUNT,RES_NAICS5_COUNT,TOT_EMP_WORK,WORK_NAICS1_TYPE,WORK_NAICS2_TYPE,WORK_NAICS3_TYPE,WORK_NAICS4_TYPE,WORK_NAICS5_TYPE,WORK_NAICS1_COUNT,WORK_NAICS2_COUNT,WORK_NAICS3_COUNT,WORK_NAICS4_COUNT,WORK_NAICS5_COUNT,RES_CITY1_TYPE,RES_CITY2_TYPE,RES_CITY3_TYPE,RES_CITY4_TYPE,RES_CITY5_TYPE,RES_CITY1_COUNT,RES_CITY2_COUNT,RES_CITY3_COUNT,RES_CITY4_COUNT,RES_CITY5_COUNT,WORK_CITY1_TYPE,WORK_CITY2_TYPE,WORK_CITY3_TYPE,WORK_CITY4_TYPE,WORK_CITY5_TYPE,WORK_CITY1_COUNT,WORK_CITY2_COUNT,WORK_CITY3_COUNT,WORK_CITY4_COUNT,WORK_CITY5_COUNT,MedIncFam_Hcost,MedIncFam_Tcost,MedIncFam_HTcost,Ret_Hcost,Ret_Tcost,Ret_HTcost,SPFam_Hcost,SPFam_Tcost,SPFam_HTcost,ModIncFam_Hcost,ModIncFam_Tcost,ModIncFam_HTcost,IDES_START_EMP,IDES_MID_EMP,IDES_CURR_EMP,AVG_VMT,RET_SALES,RES_EAV,CMRCL_EAV,IND_EAV,RAIL_EAV,FARM_EAV,MIN_EAV,TOT_EAV,WALKSCORE,OPEN_SPACE_PER_1000,TOT_ACRES,SF,Sfperc,MF,Mfperc,MIX,MIXperc,COMM,COMMperc,INST,INSTperc,IND,INDperc,TRANS,TRANSperc,AG,Agperc,OPEN,OPENperc,VACANT,VACperc,CT_1PHH,CT_2PHH,CT_3PHH,CT_4MPHH,CT_FAM_HH,CT_SP_WCHILD,CT_NONFAM_HH,HCUND20K,HCUND20K_LT20PCT,HCUND20K_20_29PCT,HCUND20K_30MPCT,HC20Kto49K,HC20Kto49K_LT20PCT,HC20Kto49K_20_29PCT,HC20Kto49K_30MPCT,HC50Kto75K,HC50Kto75K_LT20PCT,HC50Kto75K_20_29PCT,HC50Kto75K_30MPCT,HCOV75K,HCOV75K_LT20PCT,HCOV75K_20_29PCT,HCOV75K_30MPCT,NATIVE,FOR_BORN,POP_OV5,ONLY_ENGLISH,NOT_ENGLISH,LING_ISO,ENGLISH,SPANISH,POLISH,CHINESE,TAGALOG,ARABIC,KOREAN,OTHER_ASIAN,OTHER_EURO,OTHER_UNSPEC,2000_WHITE,2000_HISP,2000_BLACK,2000_ASIAN,2000_OTHER
Albany Park,57655,51542,51575,13354,13738,11975,8232,2329,1343,604,33.6,14932,24892,2337,7883,1531,49998,41047,29653,27243,2339,11394,1041,25729,12814,2494,8580,1622,219,NaN,3413,7459,4184,1355,34065,7378,8192,5068,2047,7339,4041,3627,3835,2967,2091,2259,1632,55561,16411,6415,9996,1685,18096,3413,226,3476,3654,7303,5.4,834,1612,2924,12726,NaN,4353,7239,4790,1336,378,17805,Health Care,Accommodation and Food Service,Retail Trade,Professional,Education,2516,1939,1886,1526,1505,4977,Health Care,Retail Trade,Manufacturing,Administration,Information,1081,784,618,534,401,Outside of Chicago,The Loop,Near North Side,Near West Side,Albany Park,7184,2563,1390,769,560,Outside of Chicago,Albany Park,Irving Park,Belmont Cragin,Portage Park,2130,560,222,154,144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.661,1229.16,302.877,0.24641,295.528,0.24043,34.521,0.02808,75.04,0.06105,53.804,0.04377,20.188,0.01642,413.624,0.33651,NaN,NaN,28.103,0.02286,5.475,0.00445,4022,4412,2725,5252,10752,1534,5659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25587,22290,47877,16365,31512,16525,16365,21012,967,294,2310,621,650,1801,3294,563,15866,26741,1907,10178,2963
Archer Heights,12644,13393,13233,4016,2933,2879,1897,907,369,231,33,2477,10178,79,439,60,13231,10012,6700,6038,662,3312,80,5742,3582,878,935,329,19,NaN,600,1557,1107,642,8151,2599,2881,1535,278,663,195,946,1149,787,384,493,148,44949,3906,2188,1719,361,4267,1904,44,1092,1019,183,5.6,24,808,2248,11

#### API Call for Start Year
* Currently using 2002 rather than 2001, because 2001 data lacks 'community_area' for most reported crimes. With more time, you could use loops and conditions on 'latitude' and 'longitude' to fill 'community area'.
* Export the data to CSV so that there is a reference file in case original data source is ever corrupted, or non-responsive to API calls.
* For more on the City of Chicago Crime Data: https://dev.socrata.com/foundry/data.cityofchicago.org/6zsd-86xi

* FIGURE OUT: how to remove decimal from neighborhood

In [4]:
results = client.get("6zsd-86xi", year=2002, limit=400000)
results_df = pd.DataFrame.from_records(results) # Convert to pandas DataFrame # results_df.head()
start_results = results_df[['community_area','fbi_code','date','description','latitude','longitude','primary_type','year']]
start_results['community_area'] = pd.to_numeric(start_results['community_area'], errors='coerce')
start_format = start_results.sort_values('community_area')
start_format.to_csv('report_reference_data/start_tfile.csv')    
start_format.head()


,community_area,fbi_code,date,description,latitude,longitude,primary_type,year
345584,0.0,06,2002-12-17T00:00:00.000,OVER $500,41.799089601,-87.790549465,THEFT,2002
30940,0.0,07,2002-02-04T19:30:00.000,AUTOMOBILE,41.881536805,-87.616595359,MOTOR VEHICLE THEFT,2002
181971,0.0,26,2002-07-05T10:30:00.000,TELEPHONE THREAT,41.798886497,-87.790543844,OTHER OFFENSE,2002
179314,0.0,14,2002-07-09T10:00:00.000,TO VEHICLE,41.693049104,-87.690994029,CRIMINAL DAMAGE,2002
261695,0.0,08B,2002-09-21T13:15:00.000,SIMPLE,41.970514812,-87.83665128,BATTERY,2002


#### Combining the Crime Data with our Research Indexes (Gentrification & FBI Crime Category)

In [5]:
baseyear_fbiindex = start_format.merge(fbi_index, left_on='fbi_code', right_on='fbi_code_key')
b_crimecat_cca = baseyear_fbiindex.merge(gent_fin, left_on='community_area', right_on='community_area')
b_crimecat_cca.to_csv('report_reference_data/b_crimecat_cca.csv') # display(b_crimecat_cca.columns)
b_crimecat_cca.head()

,community_area,fbi_code,date,description,latitude,longitude,primary_type,year,fbi_Crime_type,fbi_code_key,...,1970,1980,1990,2000,2010,1970-2010,Change,Type,Classification,Class_Code
0,1.0,26,2002-07-06T13:15:00.000,TO LAND,42.00667554,-87.66667767,CRIMINAL TRESPASS,2002,Misc Non-Index Offense,26,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
1,1.0,26,2002-09-20T12:00:00.000,TELEPHONE THREAT,42.012523216,-87.680974794,OTHER OFFENSE,2002,Misc Non-Index Offense,26,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
2,1.0,26,2002-11-08T21:59:53.000,FOUND SUSPECT NARCOTICS,42.021477909,-87.665304916,NARCOTICS,2002,Misc Non-Index Offense,26,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
3,1.0,26,2002-05-12T10:00:00.000,HARASSMENT BY TELEPHONE,42.005028086,-87.660870591,OTHER OFFENSE,2002,Misc Non-Index Offense,26,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
4,1.0,26,2002-08-11T00:27:00.000,HARASSMENT BY TELEPHONE,42.007180564,-87.670009605,OTHER OFFENSE,2002,Misc Non-Index Offense,26,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7


In [6]:
b_crimecat_cca_clean = b_crimecat_cca[['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type', 'ds_fbi_group', 'Classification', 'Class_Code',]]
b_crimecat_cca_group = b_crimecat_cca_clean.groupby(by=['ds_fbi_group','Classification']).count() #relevant_results.sort_values('fbi_code')
crime_reset = b_crimecat_cca_group.reset_index()
print("Count of FBI Crime Categories by Gentrification Pathway")
crime_reset.drop(columns=['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type',])
crime_reset.pivot(index='Classification', columns='ds_fbi_group', values='Class_Code')


Count of FBI Crime Categories by Gentrification Pathway


ds_fbi_group,Nonviolent,Violent,White Collar
Classification,,,
Extreme Poverty,20150,14383,532
Gentrification,15490,8146,2255
Middle Class,9261,4901,843
Mild Decline,4701,2619,391
Moderate Decline,12325,7757,497
Not Gentrification,3027,1762,222
Poverty,3802,2348,106
Serious Decline,24423,15809,1020
Upper Class,2217,1221,215


In [7]:
b_crimecat_cca_clean['Class_Code'] = pd.to_numeric(b_crimecat_cca_clean['Class_Code'], errors='coerce') # b_crimecat_cca_clean.dtypes
ngb1 = b_crimecat_cca_clean.loc[b_crimecat_cca_clean['Class_Code'] < 3]
ngb2 = b_crimecat_cca_clean.loc[b_crimecat_cca_clean['Class_Code'] == 5]
gb6 = b_crimecat_cca_clean.loc[b_crimecat_cca_clean['Class_Code'] == 6]
combo = [ngb1, ngb2]
nGentrified = pd.concat(combo)
nGentrified["DS_Subset"] = "Not Gentrified"
gb6["DS_Subset"] = "Gentrified"
neighborhodtype_bycrime = [nGentrified, gb6]
b_neighbors = pd.concat(neighborhodtype_bycrime)
b_neighbors_reset = b_neighbors.groupby(by=['DS_Subset','Neighborhood','ds_fbi_group']).count()
b_neighbors_dropem = b_neighbors_reset.reset_index()
b_neighbors_dropem.drop(columns=['Classification','community_area','fbi_code','fbi_Crime_type',], inplace=True)
print("Count of FBI Crime Categories by Gentrification Pathway")
b_neighbors_final = b_neighbors_dropem.groupby(by=['DS_Subset','Neighborhood','ds_fbi_group']).sum().unstack('ds_fbi_group')
base_year_ngdata = b_neighbors_final.reset_index()# .pivot(index='Neighborhood', columns='ds_fbi_group', values='Class_Code')
base_year_ngdata['Total'] = base_year_ngdata.sum(axis=1)
base_year_ngdata.to_csv('report_reference_data/close_year_ngdata.csv')
base_year_ngdata

Count of FBI Crime Categories by Gentrification Pathway


DS_Subset     Neighborhood Class_Code                       \
ds_fbi_group                                  Nonviolent Violent White Collar   
0                 Gentrified        Lake View       1383     657          263   
1                 Gentrified     Lincoln Park       1163     504          209   
2                 Gentrified             Loop       1214     588          451   
3                 Gentrified  Near North Side       2824    1552          567   
4                 Gentrified  Near South Side        694     456           55   
5                 Gentrified   Near West Side       2840    1518          213   
6                 Gentrified     North Center        664     309           70   
7                 Gentrified           Uptown       1721    1017          242   
8                 Gentrified        West Town       2987    1545          185   
9             Not Gentrified    Armour Square        262     113           15   
10            Not Gentrified          Beverly        311     160           46   
11            Not Gentrified       Bridgeport        694     429           39   
12            Not Gentrified  Calumet Heights        481     269           27   
13            Not Gentrified         Clearing        442     216           45   
14            Not Gentrified          Dunning        710     315           43   
15            Not Gentrified        Edgewater        977     568          105   
16            Not Gentrified      Edison Park        110      51            4   
17            Not Gentrified      Forest Glen        208      65           14   
18            Not Gentrified   Garfield Ridge       1036     474          151   
19            Not Gentrified        Hyde Park        494     282           71   
20            Not Gentrified      Irving Park       1072     569           68   
21            Not Gentrified   Jefferson Park        405     208           53   
22            Not Gentrified          Kenwood        447     417           36   
23            Not Gentrified   Lincoln Square        671     419           69   
24            Not Gentrified     Logan Square       2071    1220          168   
25            Not Gentrified      Morgan Park        824     483           42   
26            Not Gentrified  Mount Greenwood        248     135           27   
27            Not Gentrified       North Park        251     118           23   
28            Not Gentrified     Norwood Park        436     223           39   
29            Not Gentrified     Portage Park       1357     597           98   
30            Not Gentrified       West Ridge        998     553           97   

             Total  
ds_fbi_group        
0             2303  
1             1876  
2             2253  
3             4943  
4             1205  
5             4571  
6             1043  
7             2980  
8             4717  
9              390  
10             517  
11            1162  
12             777  
13             703  
14            1068  
15            1650  
16             165  
17             287  
18            1661  
19             847  
20            1709  
21             666  
22             900  
23            1159  
24            3459  
25            1349  
26             410  
27             392  
28             698  
29            2052  
30            1648

In [8]:
midresults = client.get("6zsd-86xi", year=2010, limit=400000)
midresults_df = pd.DataFrame.from_records(midresults) # Convert to pandas DataFrame # results_df.head()
mid_results = midresults_df[['community_area','fbi_code','date','description','latitude','longitude','primary_type','year']]
mid_results['community_area'] = pd.to_numeric(mid_results['community_area'], errors='coerce')
mid_results.groupby(by=['fbi_code']).count() #relevant_results.sort_values('fbi_code')
mid_results.sort_values(['community_area'])
mid_results.to_csv('report_reference_data/mid_tfile.csv')    
mid_results.head()

,community_area,fbi_code,date,description,latitude,longitude,primary_type,year
0,71.0,02,2010-01-01T00:01:00.000,PREDATORY,NaN,NaN,CRIM SEXUAL ASSAULT,2010
1,41.0,11,2010-01-01T12:00:00.000,FINANCIAL IDENTITY THEFT $300 AND UNDER,NaN,NaN,DECEPTIVE PRACTICE,2010
2,11.0,02,2010-01-01T00:00:00.000,SEX ASSLT OF CHILD BY FAM MBR,NaN,NaN,OFFENSE INVOLVING CHILDREN,2010
3,67.0,11,2010-01-11T18:20:00.000,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,NaN,DECEPTIVE PRACTICE,2010
4,66.0,02,2010-05-01T00:00:00.000,AGGRAVATED: OTHER,NaN,NaN,CRIM SEXUAL ASSAULT,2010


In [9]:
midyear_fbiindex = mid_results.merge(fbi_index, left_on='fbi_code', right_on='fbi_code_key')
mid_crimecat_cca = midyear_fbiindex.merge(gent_fin, left_on='community_area', right_on='community_area')
mid_crimecat_cca.to_csv('report_reference_data/mid_crimecat_cca.csv') # display(b_crimecat_cca.columns)
display(mid_crimecat_cca.head())

mid_crimecat_cca_clean = mid_crimecat_cca[['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type', 'ds_fbi_group', 'Classification', 'Class_Code',]]
mid_crimecat_cca_group = mid_crimecat_cca_clean.groupby(by=['ds_fbi_group','Classification']).count() #relevant_results.sort_values('fbi_code')
mid_crime_reset = mid_crimecat_cca_group.reset_index()
print("Count of FBI Crime Categories by Gentrification Pathway")
mid_crime_reset.drop(columns=['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type',])
mid_crime_reset.pivot(index='Classification', columns='ds_fbi_group', values='Class_Code')




,community_area,fbi_code,date,description,latitude,longitude,primary_type,year,fbi_Crime_type,fbi_code_key,...,1970,1980,1990,2000,2010,1970-2010,Change,Type,Classification,Class_Code
0,41.0,11,2010-01-01T12:00:00.000,FINANCIAL IDENTITY THEFT $300 AND UNDER,NaN,NaN,DECEPTIVE PRACTICE,2010,Fraud,11,...,9,9,11,7,7,-2,No Change,2 - Middle Class,Middle Class,2
1,41.0,11,2010-04-28T12:00:00.000,FINANCIAL IDENTITY THEFT $300 AND UNDER,NaN,NaN,DECEPTIVE PRACTICE,2010,Fraud,11,...,9,9,11,7,7,-2,No Change,2 - Middle Class,Middle Class,2
2,41.0,11,2010-02-17T11:15:00.000,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,NaN,DECEPTIVE PRACTICE,2010,Fraud,11,...,9,9,11,7,7,-2,No Change,2 - Middle Class,Middle Class,2
3,41.0,11,2010-07-15T12:00:00.000,FRAUD OR CONFIDENCE GAME,41.802505287,-87.585663616,DECEPTIVE PRACTICE,2010,Fraud,11,...,9,9,11,7,7,-2,No Change,2 - Middle Class,Middle Class,2
4,41.0,11,2010-07-18T00:00:00.000,FRAUD OR CONFIDENCE GAME,41.793300208,-87.594025302,DECEPTIVE PRACTICE,2010,Fraud,11,...,9,9,11,7,7,-2,No Change,2 - Middle Class,Middle Class,2


Count of FBI Crime Categories by Gentrification Pathway


ds_fbi_group,Nonviolent,Violent,White Collar
Classification,,,
Extreme Poverty,28259,17017,1394
Gentrification,16213,8943,3499
Middle Class,12208,6505,1327
Mild Decline,6442,3239,532
Moderate Decline,18074,10014,971
Not Gentrification,3594,1994,300
Poverty,4791,2757,199
Serious Decline,37405,20444,2163
Upper Class,2706,1470,343


In [10]:
mid_crimecat_cca_clean['Class_Code'] = pd.to_numeric(mid_crimecat_cca_clean['Class_Code'], errors='coerce') # b_crimecat_cca_clean.dtypes
ngm1 = mid_crimecat_cca_clean.loc[mid_crimecat_cca_clean['Class_Code'] < 3]
ngm2 = mid_crimecat_cca_clean.loc[mid_crimecat_cca_clean['Class_Code'] == 5]
gm6 = mid_crimecat_cca_clean.loc[mid_crimecat_cca_clean['Class_Code'] == 6]
combo = [ngm1, ngm2]
nGentrifiedm = pd.concat(combo)
nGentrifiedm["DS_Subset"] = "Not Gentrified"
gm6["DS_Subset"] = "Gentrified"
mid_neighborhodtype_bycrime = [nGentrifiedm, gm6]
mid_neighbors = pd.concat(mid_neighborhodtype_bycrime)
mid_neighbors_reset = mid_neighbors.groupby(by=['DS_Subset','Neighborhood','ds_fbi_group']).count()
mid_neighbors_dropem = mid_neighbors_reset.reset_index()
mid_neighbors_dropem.drop(columns=['Classification','community_area','fbi_code','fbi_Crime_type',], inplace=True)
mid_neighbors_dropem



,DS_Subset,Neighborhood,ds_fbi_group,Class_Code
0,Gentrified,Lake View,Nonviolent,1744
1,Gentrified,Lake View,Violent,954
2,Gentrified,Lake View,White Collar,529
3,Gentrified,Lincoln Park,Nonviolent,1292
4,Gentrified,Lincoln Park,Violent,551
5,Gentrified,Lincoln Park,White Collar,294
6,Gentrified,Loop,Nonviolent,1612
7,Gentrified,Loop,Violent,831
8,Gentrified,Loop,White Collar,698
9,Gentrified,Near North Side,Nonviolent,2800


In [11]:
print("Count of FBI Crime Categories by Gentrification Pathway")
mid_neighbors_final = mid_neighbors_dropem.groupby(by=['DS_Subset','Neighborhood','ds_fbi_group']).sum().unstack('ds_fbi_group')
mid_year_ngdata = mid_neighbors_final.reset_index()# .pivot(index='Neighborhood', columns='ds_fbi_group', values='Class_Code')
mid_year_ngdata['Total'] = mid_year_ngdata.sum(axis=1)
mid_year_ngdata.to_csv('report_reference_data/close_year_ngdata.csv')
mid_year_ngdata

Count of FBI Crime Categories by Gentrification Pathway


DS_Subset     Neighborhood Class_Code                       \
ds_fbi_group                                  Nonviolent Violent White Collar   
0                 Gentrified        Lake View       1744     954          529   
1                 Gentrified     Lincoln Park       1292     551          294   
2                 Gentrified             Loop       1612     831          698   
3                 Gentrified  Near North Side       2800    1648          866   
4                 Gentrified  Near South Side        562     354           87   
5                 Gentrified   Near West Side       2780    1639          322   
6                 Gentrified     North Center        594     233           96   
7                 Gentrified           Uptown       1769    1000          240   
8                 Gentrified        West Town       3060    1733          367   
9             Not Gentrified    Armour Square        445     222           45   
10            Not Gentrified          Beverly        400     182           93   
11            Not Gentrified       Bridgeport        890     447           57   
12            Not Gentrified  Calumet Heights        632     369           66   
13            Not Gentrified         Clearing        552     300           34   
14            Not Gentrified          Dunning        868     421          101   
15            Not Gentrified        Edgewater       1133     726          162   
16            Not Gentrified      Edison Park        116      74           17   
17            Not Gentrified      Forest Glen        242      84           34   
18            Not Gentrified   Garfield Ridge       1058     439          137   
19            Not Gentrified        Hyde Park        575     353           94   
20            Not Gentrified      Irving Park       1432     833          122   
21            Not Gentrified   Jefferson Park        553     267           79   
22            Not Gentrified          Kenwood        759     532           55   
23            Not Gentrified   Lincoln Square        842     427           93   
24            Not Gentrified     Logan Square       2259    1325          198   
25            Not Gentrified      Morgan Park       1172     625          109   
26            Not Gentrified  Mount Greenwood        367     175           49   
27            Not Gentrified       North Park        373     159           37   
28            Not Gentrified     Norwood Park        478     279           99   
29            Not Gentrified     Portage Park       1676     873          168   
30            Not Gentrified       West Ridge       1686     857          121   

             Total  
ds_fbi_group        
0             3227  
1             2137  
2             3141  
3             5314  
4             1003  
5             4741  
6              923  
7             3009  
8             5160  
9              712  
10             675  
11            1394  
12            1067  
13             886  
14            1390  
15            2021  
16             207  
17             360  
18            1634  
19            1022  
20            2387  
21             899  
22            1346  
23            1362  
24            3782  
25            1906  
26             591  
27             569  
28             856  
29            2717  
30            2664

In [12]:
closeresults = client.get("6zsd-86xi", year=2018, limit=400000)
closeresults_df = pd.DataFrame.from_records(closeresults)                    # Convert to pandas DataFrame # results_df.head()
close_results = closeresults_df[['community_area','fbi_code','date','description', 'latitude','longitude','primary_type','year']]
close_results['community_area'] = pd.to_numeric(close_results['community_area'], errors='coerce')
close_results.groupby(by=['fbi_code']).count() #relevant_results.sort_values('fbi_code')
close_results.to_csv('report_reference_data/close_tfile.csv')    
close_results.head()


,community_area,fbi_code,date,description,latitude,longitude,primary_type,year
0,53.0,01A,2018-02-04T01:36:00.000,FIRST DEGREE MURDER,41.68073915,-87.645661144,HOMICIDE,2018
1,75.0,08B,2018-01-07T21:01:00.000,DOMESTIC BATTERY SIMPLE,41.692258729,-87.65053259,BATTERY,2018
2,8.0,06,2018-01-05T18:10:00.000,RETAIL THEFT,41.890384236,-87.624109917,THEFT,2018
3,29.0,18,2018-01-08T08:59:00.000,MANU/DELIVER: HEROIN (WHITE),41.858283017,-87.723691271,NARCOTICS,2018
4,39.0,26,2018-01-08T10:45:00.000,HARASSMENT BY TELEPHONE,41.809297059,-87.591967028,OTHER OFFENSE,2018


In [13]:
closeyear_fbiindex = close_results.merge(fbi_index, left_on='fbi_code', right_on='fbi_code_key')
close_crimecat_cca = closeyear_fbiindex.merge(gent_fin, left_on='community_area', right_on='community_area')
close_crimecat_cca.to_csv('report_reference_data/close_crimecat_cca.csv') # display(b_crimecat_cca.columns)
display(close_crimecat_cca.head())

close_crimecat_cca_clean = close_crimecat_cca[['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type', 'ds_fbi_group', 'Classification', 'Class_Code',]]
close_crimecat_cca_group = close_crimecat_cca_clean.groupby(by=['ds_fbi_group','Classification']).count() #relevant_results.sort_values('fbi_code')
close_crime_reset = close_crimecat_cca_group.reset_index()
print("Count of FBI Crime Categories by Gentrification Pathway")
close_crime_reset.drop(columns=['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type',])
close_crime_reset.pivot(index='Classification', columns='ds_fbi_group', values='Class_Code')


,community_area,fbi_code,date,description,latitude,longitude,primary_type,year,fbi_Crime_type,fbi_code_key,...,1970,1980,1990,2000,2010,1970-2010,Change,Type,Classification,Class_Code
0,75.0,08B,2018-01-07T21:01:00.000,DOMESTIC BATTERY SIMPLE,41.692258729,-87.65053259,BATTERY,2018,Simple Battery,08B,...,5,3,3,3,1,-4,No Change,2 - Middle Class,Middle Class,2
1,75.0,08B,2018-02-11T20:30:00.000,SIMPLE,41.694700209,-87.682526799,BATTERY,2018,Simple Battery,08B,...,5,3,3,3,1,-4,No Change,2 - Middle Class,Middle Class,2
2,75.0,08B,2018-03-03T23:30:00.000,DOMESTIC BATTERY SIMPLE,41.693188884,-87.64819774,BATTERY,2018,Simple Battery,08B,...,5,3,3,3,1,-4,No Change,2 - Middle Class,Middle Class,2
3,75.0,08B,2018-04-23T11:41:00.000,DOMESTIC BATTERY SIMPLE,41.693258507,-87.642408263,BATTERY,2018,Simple Battery,08B,...,5,3,3,3,1,-4,No Change,2 - Middle Class,Middle Class,2
4,75.0,08B,2018-04-16T01:00:00.000,SIMPLE,41.694080673,-87.650945376,BATTERY,2018,Simple Battery,08B,...,5,3,3,3,1,-4,No Change,2 - Middle Class,Middle Class,2


Count of FBI Crime Categories by Gentrification Pathway


ds_fbi_group,Nonviolent,Violent,White Collar
Classification,,,
Extreme Poverty,18035,12625,1767
Gentrification,10075,8671,6660
Middle Class,6914,5058,1923
Mild Decline,3259,2579,811
Moderate Decline,10101,6847,1288
Not Gentrification,1745,1303,544
Poverty,2696,2307,448
Serious Decline,19505,16037,3064
Upper Class,1539,1264,626


In [14]:
close_crimecat_cca_clean['Class_Code'] = pd.to_numeric(close_crimecat_cca_clean['Class_Code'], errors='coerce') # b_crimecat_cca_clean.dtypes
ngc1 = close_crimecat_cca_clean.loc[close_crimecat_cca_clean['Class_Code'] < 3]
ngc2 = close_crimecat_cca_clean.loc[close_crimecat_cca_clean['Class_Code'] == 5]
gc6 = close_crimecat_cca_clean.loc[close_crimecat_cca_clean['Class_Code'] == 6]
combo = [ngc1, ngc2]
nGentrifiedc = pd.concat(combo)
nGentrifiedc["DS_Subset"] = "Not Gentrified"
gc6["DS_Subset"] = "Gentrified"
close_neighborhodtype_bycrime = [nGentrifiedc, gc6]
close_neighbors = pd.concat(close_neighborhodtype_bycrime)
close_neighbors_reset = close_neighbors.groupby(by=['DS_Subset','Neighborhood','ds_fbi_group']).count()
close_neighbors_dropem = close_neighbors_reset.reset_index()
close_neighbors_dropem.drop(columns=['Classification','community_area','fbi_code','fbi_Crime_type',], inplace=True)
print("Count of FBI Crime Categories by Gentrification Pathway")
close_neighbors_final = close_neighbors_dropem.groupby(by=['DS_Subset','Neighborhood','ds_fbi_group']).sum().unstack('ds_fbi_group')
close_year_ngdata = close_neighbors_final.reset_index()# .pivot(index='Neighborhood', columns='ds_fbi_group', values='Class_Code')
close_year_ngdata['Total'] = close_year_ngdata.sum(axis=1)
close_year_ngdata.to_csv('report_reference_data/close_year_ngdata.csv')
close_year_ngdata               
                         

Count of FBI Crime Categories by Gentrification Pathway


DS_Subset     Neighborhood Class_Code                       \
ds_fbi_group                                  Nonviolent Violent White Collar   
0                 Gentrified        Lake View       1055     929          693   
1                 Gentrified     Lincoln Park        863     475          439   
2                 Gentrified             Loop       1409    1318         1557   
3                 Gentrified  Near North Side       1784    1849         1929   
4                 Gentrified  Near South Side        432     362          237   
5                 Gentrified   Near West Side       1940    1620          772   
6                 Gentrified     North Center        292     165          127   
7                 Gentrified           Uptown        834     858          354   
8                 Gentrified        West Town       1466    1095          552   
9             Not Gentrified    Armour Square        199     207           75   
10            Not Gentrified          Beverly        285     147           93   
11            Not Gentrified       Bridgeport        460     278          124   
12            Not Gentrified  Calumet Heights        370     348           95   
13            Not Gentrified         Clearing        297     196           92   
14            Not Gentrified          Dunning        423     370          112   
15            Not Gentrified        Edgewater        633     585          269   
16            Not Gentrified      Edison Park         70      72           26   
17            Not Gentrified      Forest Glen        118      59           57   
18            Not Gentrified   Garfield Ridge        712     326          194   
19            Not Gentrified        Hyde Park        492     264          176   
20            Not Gentrified      Irving Park        702     575          173   
21            Not Gentrified   Jefferson Park        326     214           87   
22            Not Gentrified          Kenwood        382     307          115   
23            Not Gentrified   Lincoln Square        469     391          192   
24            Not Gentrified     Logan Square       1086     818          345   
25            Not Gentrified      Morgan Park        631     484          140   
26            Not Gentrified  Mount Greenwood        186     104           60   
27            Not Gentrified       North Park        249     157           82   
28            Not Gentrified     Norwood Park        324     258          106   
29            Not Gentrified     Portage Park        847     710          228   
30            Not Gentrified       West Ridge        937     755          252   

             Total  
ds_fbi_group        
0             2677  
1             1777  
2             4284  
3             5562  
4             1031  
5             4332  
6              584  
7             2046  
8             3113  
9              481  
10             525  
11             862  
12             813  
13             585  
14             905  
15            1487  
16             168  
17             234  
18            1232  
19             932  
20            1450  
21             627  
22             804  
23            1052  
24            2249  
25            1255  
26             350  
27             488  
28             688  
29            1785  
30            1944

In [15]:
pop_table = cca_codex[['GEOG', '2000_POP','2010_POP','POP_CHANGE']]
# display(cca_codex.head())

pop_table['02_Change_est'] = round(pop_table['POP_CHANGE'] / 5)
pop_table['2002_POP_est'] = pop_table['2000_POP'] + pop_table['02_Change_est']

pop_index = pop_table[['GEOG', '2000_POP','2002_POP_est','2010_POP']]
pop_index['GEOG'] = pop_index['GEOG'].str.replace("The Loop","Loop")


# list(pop_index['GEOG'])

In [18]:
cca_codex2018.columns

Index(['<<<<<<< HEAD:Resources/DemoData2018.csv'], dtype='object')

In [17]:
cca_codex2018_s = cca_codex2018[["GEOG", "TOT_POP"]]
cca_codex2018_s['GEOG'] = cca_codex2018_s['GEOG'].str.replace("The Loop","Loop")
cca_codex2018_name = cca_codex2018_s.rename(columns = {"GEOG": "Neighborhood", "TOT_POP" : "2018_POP"})
cca_codex2018_name.head()

KeyError: "['GEOG' 'TOT_POP'] not in index"

In [ ]:
base_name = base_year_ngdata.rename(columns={"Nonviolent": "Nonviolent_2002", "Violent": "Violent_2002", "White Collar": "White_Collar_2002", "Total":"Total_2002"})
midmerge = mid_year_ngdata.drop(columns=['DS_Subset'])
mid_name = midmerge.rename(columns={"Nonviolent": "Nonviolent_2010", "Violent": "Violent_2010", "White Collar": "White_Collar_2010","Total":"Total_2010"})

closemerge = close_year_ngdata.drop(columns=['DS_Subset'])
close_name = closemerge.rename(columns={"Nonviolent": "Nonviolent_2018", "Violent": "Violent_2018", "White Collar": "White_Collar_2018","Total":"Total_2018"})
#display(base_name.head()) display(mid_name.head()) display(close_name.head())

twoandten = base_name.merge(mid_name, how='left', on='Neighborhood')
twotenandeighteen = twoandten.merge(close_name, how='left', on='Neighborhood')
pop_index_allyr = pop_index.merge(cca_codex2018_name, how='left',  left_on='GEOG', right_on='Neighborhood')
pop_index_allyr_c = pop_index_allyr.drop(columns=['Neighborhood'])

crime_all_yr_merge = twotenandeighteen.merge(pop_index_allyr_c, how='left', left_on='Neighborhood', right_on='GEOG')
crime_all_yr_merge.dtypes


In [ ]:
crime_all_yr_name = crime_all_yr_merge.rename(columns=
                                              {list(crime_all_yr_merge)[0]: "Gentrified_type", list(crime_all_yr_merge)[1]: "Neighborhood", 
                                               list(crime_all_yr_merge)[2]: "Nonviolent_2002", list(crime_all_yr_merge)[3]: "Violent_2002", 
                                               list(crime_all_yr_merge)[4]: "White_Collar_2002",list(crime_all_yr_merge)[5]: "Total_2002", 
                                               list(crime_all_yr_merge)[6]: "Nonviolent_2010", list(crime_all_yr_merge)[7]: "Violent_2010",
                                               list(crime_all_yr_merge)[8]: "White_Collar_2010", list(crime_all_yr_merge)[9]: "Total_2010",
                                               list(crime_all_yr_merge)[10]: "Nonviolent_2018", list(crime_all_yr_merge)[11]: "Violent_2018",
                                               list(crime_all_yr_merge)[12]: "White_Collar_2018", list(crime_all_yr_merge)[13]: "Total_2018",})

crime_all_yr_name['Nonviolent_2002_est_rate'] = (crime_all_yr_name['Nonviolent_2002'] / crime_all_yr_name['2002_POP_est']) * 1000
crime_all_yr_name['Violent_2002_est_rate'] = (crime_all_yr_name['Violent_2002'] / crime_all_yr_name['2002_POP_est']) * 1000
crime_all_yr_name['White_Collar_2002_est_rate'] = (crime_all_yr_name['White_Collar_2002'] / crime_all_yr_name['2002_POP_est']) * 1000
crime_all_yr_name['Total_2002_est_rate'] = (crime_all_yr_name['Total_2002'] / crime_all_yr_name['2002_POP_est']) * 1000
crime_all_yr_name['Nonviolent_2010_rate'] = (crime_all_yr_name['Nonviolent_2010'] / crime_all_yr_name['2010_POP']) * 1000
crime_all_yr_name['Violent_2010_rate'] = (crime_all_yr_name['Violent_2010'] / crime_all_yr_name['2010_POP']) * 1000
crime_all_yr_name['White_Collar_2010_rate'] = (crime_all_yr_name['White_Collar_2010'] / crime_all_yr_name['2010_POP']) * 1000
crime_all_yr_name['Total_2010_rate'] = (crime_all_yr_name['Total_2010'] / crime_all_yr_name['2010_POP']) * 1000

crime_all_yr_name['Nonviolent_2018_rate'] = (crime_all_yr_name['Nonviolent_2018'] / crime_all_yr_name['2018_POP']) * 1000
crime_all_yr_name['Violent_2018_rate'] = (crime_all_yr_name['Violent_2018'] / crime_all_yr_name['2018_POP']) * 1000
crime_all_yr_name['White_Collar_2018_rate'] = (crime_all_yr_name['White_Collar_2018'] / crime_all_yr_name['2018_POP']) * 1000
crime_all_yr_name['Total_2018_rate'] = (crime_all_yr_name['Total_2018'] / crime_all_yr_name['2018_POP']) * 1000

crime_all_yr_name.to_csv('report_reference_data/crimeby_neighborhood_years_count.csv')
display(crime_all_yr_name.head(30))

In [ ]:
crime_all_yr = twotenandeighteen.iloc[:,[0,1,2,6,10,3,7,11,4,8,12]]

gentrified_crime = twotenandeighteen[twotenandeighteen['DS_Subset'] == 'Gentrified']
not_gentrified_crime = twotenandeighteen[twotenandeighteen['DS_Subset'] == 'Not Gentrified']

gentrified_nonviolent_yr = gentrified_crime.iloc[:,[0,1,2,6,10]]
gentrified_violent_yr = gentrified_crime.iloc[:,[0,1,3,7,11]]
gentrified_whitecollar_yr = gentrified_crime.iloc[:,[0,1,4,8,12]]
                                    
not_gentrified_nonviolent_yr = not_gentrified_crime.iloc[:,[0,1,2,6,10]]
not_gentrified_violent_yr = not_gentrified_crime.iloc[:,[0,1,3,7,11]]
not_gentrified_whitecollar_yr = not_gentrified_crime.iloc[:,[0,1,4,8,12]]

display(crime_all_yr)
display(gentrified_nonviolent_yr)
display(not_gentrified_nonviolent_yr)
display(gentrified_violent_yr)
display(not_gentrified_violent_yr)
display(gentrified_whitecollar_yr)
display(not_gentrified_whitecollar_yr)

display(gentrified_crime) 
display(not_gentrified_crime) 

In [ ]:
gentrified_crime_rate = crime_all_yr_name[crime_all_yr_name['Gentrified_type'] == 'Gentrified']
not_gentrified_crime_rate = crime_all_yr_name[crime_all_yr_name['Gentrified_type'] == 'Not Gentrified']

display(gentrified_crime_rate)
display(not_gentrified_crime_rate)

In [ ]:
crimerate_nonviolent_g = pd.melt(gentrified_crime_rate, id_vars=['Gentrified_type'], value_vars=['Nonviolent_2002_est_rate','Nonviolent_2010_rate','Nonviolent_2018_rate'])
crimerate_violent_g = pd.melt(gentrified_crime_rate, id_vars=['Gentrified_type'], value_vars=['Violent_2002_est_rate','Violent_2010_rate','Violent_2018_rate'])
crimerate_white_collar_g = pd.melt(gentrified_crime_rate, id_vars=['Gentrified_type'], value_vars=['White_Collar_2002_est_rate','White_Collar_2010_rate','White_Collar_2018_rate'])

crimerate_nonviolent_ng = pd.melt(not_gentrified_crime_rate, id_vars=['Gentrified_type'], value_vars=['Nonviolent_2002_est_rate','Nonviolent_2010_rate','Nonviolent_2018_rate'])
crimerate_violent_ng = pd.melt(not_gentrified_crime_rate, id_vars=['Gentrified_type'], value_vars=['Violent_2002_est_rate','Violent_2010_rate','Violent_2018_rate'])
crimerate_white_collar_ng = pd.melt(not_gentrified_crime_rate, id_vars=['Gentrified_type'], value_vars=['White_Collar_2002_est_rate','White_Collar_2010_rate','White_Collar_2018_rate'])
crimerate_nonviolent_g


In [ ]:
crimerate_nonviolent_g.boxplot("value", by="variable", figsize=(20, 10))

plt.title('Nonviolent Crime Rate in Gentrified Neighborhoods', fontsize=20)
plt.suptitle("")

plt.xlabel('Year', fontsize=20)
plt.ylabel('Crime Rate (per 1,000)', fontsize=20)
 
# Limits for the Y axis
bars = ('','2002 est', '2010', '2018','')
y_pos = np.arange(len(bars))

 
# Create names
plt.xticks(y_pos, bars, fontsize=14)
plt.ylim(top=70)

# Export graphic
plt.savefig('output_reports_figures/gentrified_nonviolent.png')

In [ ]:
crimerate_violent_g.boxplot("value", by="variable", figsize=(20, 10))

plt.title('Violent Crime Rate in Gentrified Neighborhoods', fontsize=20)
plt.suptitle("")

plt.xlabel('Year', fontsize=20)
plt.ylabel('Crime Rate (per 1,000)', fontsize=20)
 
# Limits for the Y axis
bars = ('','2002 est', '2010', '2018','')
y_pos = np.arange(len(bars))

 
# Create names
plt.xticks(y_pos, bars, fontsize=14)
plt.ylim(top=50)
 
# Export graphic
plt.savefig('output_reports_figures/gentrified_violent.png')

In [ ]:
crimerate_white_collar_g.boxplot("value", by="variable", figsize=(20, 10))

plt.title('White Collar Crime Rate in Gentrified Neighborhoods', fontsize=20)
plt.suptitle("")

plt.xlabel('Year', fontsize=20)
plt.ylabel('Crime Rate (per 1,000)', fontsize=20)
 
# Limits for the Y axis
bars = ('','2002 est', '2010', '2018','')
y_pos = np.arange(len(bars))

 
# Create names
plt.xticks(y_pos, bars, fontsize=14)
plt.ylim(top=50)

# Export graphic
plt.savefig('output_reports_figures/gentrified_whitecollar.png')

In [ ]:
crimerate_nonviolent_ng.boxplot("value", by="variable", figsize=(20, 10))

plt.title('Nonviolent Crime Rate in Non-Gentrified Neighborhoods', fontsize=20)
plt.suptitle("")

plt.xlabel('Year', fontsize=20)
plt.ylabel('Crime Rate (per 1,000)', fontsize=20)
 
# Limits for the Y axis
bars = ('','2002 est', '2010', '2018','')
y_pos = np.arange(len(bars))

 
# Create names
plt.xticks(y_pos, bars, fontsize=14)
plt.ylim(top=50)

# Export graphic
plt.savefig('output_reports_figures/nongentrified_nonviolent.png')

In [ ]:
crimerate_violent_ng.boxplot("value", by="variable", figsize=(20, 10))

plt.title('Violent Crime Rate in Non-Gentrified Neighborhoods', fontsize=20)
plt.suptitle("")

plt.xlabel('Year', fontsize=20)
plt.ylabel('Crime Rate (per 1,000)', fontsize=20)
 
# Limits for the Y axis
bars = ('','2002 est', '2010', '2018','')
y_pos = np.arange(len(bars))

 
# Create names
plt.xticks(y_pos, bars, fontsize=14)
plt.ylim(top=50)

# Export graphic
plt.savefig('output_reports_figures/nongentrified_violent.png')

In [ ]:
crimerate_white_collar_ng.boxplot("value", by="variable", figsize=(20, 10))

plt.title('White Collar Crime Rate in Non-Gentrified Neighborhoods', fontsize=20)
plt.suptitle("")

plt.xlabel('Year', fontsize=20)
plt.ylabel('Crime Rate (per 1,000)', fontsize=20)
 
# Limits for the Y axis
bars = ('','2002 est', '2010', '2018','')
y_pos = np.arange(len(bars))

 
# Create names
plt.xticks(y_pos, bars)
plt.ylim(top=50)

# Export graphic
plt.savefig('output_reports_figures/nongentrified_whitecollar.png')


In [ ]:
crimerate_all_yr = crime_all_yr_name.iloc[:,[0,1,16,17,18,19,23,27,20,24,28,21,25,29]]

#display(crime_all_yr_name.columns[[0,1,18,22,19,23,20,24,21,25]])
display(crimerate_all_yr)